In [1]:
import importlib
import networks.network_trainer as ntr
import pandas as pd
network = pd.read_csv("csv/network_information.csv")
importlib.reload(ntr)

<module 'networks.network_trainer' from '/home/yuna/Documents/inference_and_learning/inference_and_learning_final_project/networks/network_trainer.py'>

In [2]:
table2 = pd.read_csv("csv/table2.csv")
for line in range(table2.__len__()):
    for data in ["CIFAR-10","CIFAR-100","SVHN"] :
        if(table2.loc[line,data] == "NaN"):
            continue
        network_data = network.loc[
            (network["use"]=="table2") &
            (network["model"]==table2.loc[line,"Model"]) &
            (network["training_method"]==table2.loc[line,"Training_Method"]) &
            (network["dataset"]==data)].iloc[0]
        net =  ntr.NetworkTrainer(network_data)
        print(net.__str__())
        net.train()
        table2.loc[line,data] = str(net.getAccuracy()) + "%"
        print("Accuracy :", net.getAccuracy(), "%")
        table2.to_csv("csv/table2.csv",index=False)




Files already downloaded and verified
Files already downloaded and verified
Network Trainer: S-CONV,SGD,CIFAR-10 -> S-CONV_SGD_CIFAR-10_512_400
¤¤¤¤¤¤¤¤¤¤|¤¤¤¤¤

KeyboardInterrupt: 